# Objectives

Todo:

- get phi polytope G4.

- get phi inequality data by cuts.
- get phi polytope for all examples.
- get center phi for G4M2, then test for other graphs.

Later:
    - consider phi meeting notes and translates.
    - get phi inequality data by cuts.

- What do we know from the old folders:
    - Taking average fails for G6 up 
    - how to get hyperplanes
    - Break divisors always give a phi.
    - We investigated the tree-phi-polytope at one point

# Focus Functions

In [1]:
import numpy as np
import itertools
import pickle

In [2]:
def unpickle(graphname):
    # Unpickle the list
    filename=f"examples/{graphname}/{graphname}.pkl"
    with open(filename, 'rb') as f:
        unpickled_list = pickle.load(f)
    return unpickled_list

In [3]:
def get_formatted_hyperplanes_txt(lbm,phi_G):#Hyperplanes #put into format of sage b<Ax<b+1
    
    """
    See phi polytope higher case doc
    
    Purpose: Get list of inquality data to feed into Sages Polyhedron function
    https://doc.sagemath.org/html/en/reference/discrete_geometry/sage/geometry/polyhedron/constructor.html
    
    Of the form: This is the format we will put our data into!
    
    Polyhedron(ieqs=[(0,1,0),(0,0,1),(1,-1,-1)]).Hrepresentation()
    (An inequality (-1, -1) x + 1 >= 0,
     An inequality (1, 0) x + 0 >= 0,
     An inequality (0, 1) x + 0 >= 0)
    
    Inputs:
    phi_G:  records indexs of cuts with cutfraction in inequalities. 
    i.e. phi_G62={"1":1,"2":1.5,"3":1,"4":1,"5":1,"6":1.5,"12":1.5,"23":1.5,"34":1,"45":1,"56":1.5,"61":1.5,"123":1.5,"234":1.5,"345":1,"456":1.5,"561":1.5,"612":1}
    lbm= line bundle mulidegrees given by chip-adding
    """
    
    #Length of array
    n=len(lbm[0])
    
    # We add the total degree equation to inequals: need both as \le and \ge format.
    deg=sum(lbm[0])
    tot_equ1=[-deg,]+ [1]*n
    tot_equ2=[deg,]+ [-1]*n
    
    #We now build all inequalities using lbm and phi_G
    inequals=[tot_equ1,tot_equ2]
    for d in lbm: #creates row at a time
        """
        Each key,value pair in phi_G give two phi inequalites.
        We iterate through pairs and record.
        """
        for indexor in list(phi_G.keys()):
            """
            We have now fixed d a lbm,indexor i.e. "134", for the phi inequalites.
            
             Polyhedron(ieqs=[(0,1,0),(0,0,1),(1,-1,-1)]).Hrepresentation()
            (An inequality (-1, -1) x + 1 >= 0,
            """
            
            def sum_comps(comp,indexor): #Done
                """returns the sum of the components of comp in the position by indexor"""
                #for d and phi
                sum_l=[comp[int(i)] for i in indexor] # 1 in indexor corrsponds to position 0 in list ect, 2 cor to 1
                return sum(sum_l)
            
            #Take the cut fraction
            cf=phi_G[indexor] # want value
            
            #Sum of lbm components of the terms given by indexor.
            constant_indx= sum_comps(d,indexor)        
        
            #position values for indexors in a list
            l_i=None # [ [int(x)-1,] for x in indexor] #x,y,z.. terms for indexors in Rf space #Indexs for phi keys
            

            def lower_part(constant_indx,l_i,indexor,cf,n):
                """
                Remember lower part we have the inequality the CORRECT way round.
                
                See the difference of l between lower and upper parts.
                """
                
                bterm=[constant_indx+cf] # as list
                
                #Start building inequality in format of description above.
                base=np.zeros(n)
                
                #position values for indexors in a list the free variable in hyperplane inequal
                l=[(-1,i) for i in indexor] # put -1 in the ith position given be indexor.
                
                #Add to base the free variables
                for item in l:
                    ipos = int(item[1]) # ith position
                    val=item[0]
                    base[ipos]=val
                
                #Add constant
                xyz=base
                ineq=bterm+list(xyz) # [b,x,y,z...] want to concat
                
                return ineq

            def upper_part(constant_indx,l_i,indexor,cf,n):
                """
                Remember upper part we have the inequality the OTHER way round. see l.
                """
                
                #Remember upper part of modulus inequality.

                bterm=[-constant_indx+cf] # as list
                base=np.zeros(n)
                l=[(1,i) for i in indexor] # put -1 in the ith position given be indexor.
                
                for item in l:
                    ipos = int(item[1]) # ith position
                    val=item[0]
                    base[ipos]=val
                
                xyz=base
                
                ineq=bterm+list(xyz) # [b,x,y,z...] want to concat
                
                return ineq

            res_lower=lower_part(constant_indx,l_i,indexor,cf,n)
            
            res_upper=upper_part(constant_indx,l_i,indexor,cf,n)
            
            res_combined= [res_lower,res_upper]
            
            inequals=inequals+res_combined
        
    return inequals

In [4]:
def poly_size(data):
    Poly=Polyhedron(ieqs=data)
    dimP=dim(Poly)
#     print(f"The dimension of the polytope is: {dimP}")
    return dimP

In [5]:
def polytope_size_checker(data,phi,size):
    
    """
    Inputs:
    data: is a list of line bundle multidegrees for the weak stabiltiies that fail when phi is taken to be the average.
    phi= is the data of the cuts and the indexes taken for the phi inequalities.
    size: the size of the nondegenerate polytope (number of vertices)
    
    """
    recorder=[]
    recorder_minus=[]
    recorder_0=[]
    recorder_1belowtop=[]
    recorder_max=[]
    dimensions=[]

    counter=0
    for lbm in data:
        lbm_data=get_formatted_hyperplanes_txt(lbm,phi)
        dimp=poly_size(lbm_data)
        if dimp ==0:
            recorder.append(lbm)
        if dimp >0 and dimp<size:
            recorder_1belowtop.append(lbm)
        if dimp ==size:
            recorder_max.append(lbm)
        if dimp==-1:
            recorder_minus.append(lbm)
            
        print(counter)
        dimensions.append(dimp)
        counter+=1

    print("Total number of cases:",len(data))
    print("Empty:",len(recorder_minus))
    print("Points:",len(recorder_0))
    print("Maximal polytopes:",len(recorder_1belowtop))
    
#     print("recorder_max:",len(recorder_max)) #always empty as hyperplane.
    
    print("\n", f"Number of cases of dim: -1:{dimensions.count(-1)}, 0: {dimensions.count(0)}, 1:{dimensions.count(1)}, 2:{dimensions.count(2)}, 3:{dimensions.count(3)}, 4:{dimensions.count(4)}, 5:{dimensions.count(5)}, 6:{dimensions.count(6)}, 7:{dimensions.count(7)}")
          
    return 

In [6]:
def size_check_name(graphname,phi_graphname,size):
    """
    Inputs: 
    graphname= e.g "G4"
    phi_graphname= e.g. {"1":1,"2":1,"3":1,"4":1,"12":1,"23":1}
    Returns: 
    """    
    #loads data
    data=unpickle(graphname)
    
    # Determine size of phi-polytopes
    all_lbm=[i[1] for i in data]
    polytope_size_checker(all_lbm,phi_graphname,size)
    
    return None

# Get phi-polytope.

In this file we construct the convex polytopal region denoted $\overline{R_f}$ for FCJ.

Which are intersections of the form $$b\le Ax \le b+1.$$

We aim to determine the number of cases where $R_f = \emptyset$. 

In [7]:
#Here graphs are labled v1,..,v_n,see get_formatted_hyperplanes_txt for how handled.

#G4 examples
phi_G4={"0":1,"1":1,"2":1,"3":1,"01":1,"12":1}
phi_G4M2={"0":1.5,"1":1,"2":1.5,"3":1,"01":1.5,"12":1.5}
phi_G4M02M13={"0":1.5,"1":1.5,"2":1.5,"3":1.5,"01":2,"12":2,"02":2}

#G5 examples
# phi_G5
phi_G5M2={"0":1.5,"1":1,"2":1.5,"3":1,"4":1,"01":1.5,"12":1.5,"23":1.5,"34":1,"40":1.5}
phi_G5M02M03={"0":2,"1":1,"2":1.5,"3":1.5,"4":1,"01":2,"04":2,"12":1.5,"23":2,"34":1.5}

phi_G5M02M03M14={"0":2,"1":1.5,"2":1.5,"3":1.5,"4":1.5,"01":2.5,"02":2.5,"03":2.5,"04":2.5,"12":2,"23":2,"34":2}

#G6 examples
# phi_G6=
phi_G6M2={"0":1.5,"1":1,"2":1.5,"3":1,"4":1,"5":1,"01":1.5,"12":1.5,"23":1.5,"34":1,"45":1,"50":1.5,"012":1.5,"123":1.5,"234":1.5,"345":1,"450":1.5,"501":1.5}
phi_G6M3={"0":1.5,"1":1,"2":1,"3":1.5,"4":1,"5":1,"01":1.5,"12":1,"23":1.5,"34":1.5,"45":1,"50":1.5,"012":1.5,"123":1.5,"234":1.5,"345":1.5,"450":1.5,"501":1.5}

# K33
phi_K33={"0":1.5,"1":1.5,"2":1.5,"3":1.5,"4":1.5,"5":1.5,"03":2,"04":2,"05":2,"13":2,"14":2,"15":2,"23":2,"24":2,"25":2,"013":2.5,"014":2.5,"015":2.5,"023":2.5,"024":2.5,"025":2.5,"034":2.5,"035":2.5,"045":2.5,"123":2.5,"124":2.5,"125":2.5,"134":2.5,"135":2.5,"145":2.5,"234":2.5,"235":2.5,"245":2.5}

## G4

- How do we take lbm and cut mapping and generate phi inequalits? get_formatted_hyperplanes_txt

In [10]:
size=4

In [11]:
size_check_name("G4",phi_G4,size)

0
1
2
3
4
5
Total number of cases: 6
Empty: 0
Points: 0
Maximal polytopes: 6

 Number of cases of dim: -1:0, 0: 0, 1:0, 2:0, 3:6, 4:0, 5:0, 6:0, 7:0


In [14]:
size_check_name("G4M2",phi_G4M2,size)

0
1
2
3
4
5
Total number of cases: 6
Empty: 0
Points: 0
Maximal polytopes: 6

 Number of cases of dim: -1:0, 0: 0, 1:0, 2:0, 3:6, 4:0, 5:0, 6:0, 7:0


In [16]:
size_check_name("G4M02M13",phi_G4M02M13,size)

0
1
2
3
4
5
6
7
8
9
Total number of cases: 10
Empty: 0
Points: 0
Maximal polytopes: 10

 Number of cases of dim: -1:0, 0: 0, 1:0, 2:0, 3:10, 4:0, 5:0, 6:0, 7:0


## G5

In [17]:
size=5

In [23]:
size_check_name("G5M2",phi_G5M2,size) #?

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
Total number of cases: 24
Empty: 0
Points: 0
Maximal polytopes: 24

 Number of cases of dim: -1:0, 0: 0, 1:0, 2:0, 3:0, 4:24, 5:0, 6:0, 7:0


In [21]:
size_check_name("G5M02M03",phi_G5M02M03,size)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
Total number of cases: 24
Empty: 0
Points: 0
Maximal polytopes: 24

 Number of cases of dim: -1:0, 0: 0, 1:0, 2:0, 3:0, 4:24, 5:0, 6:0, 7:0


In [25]:
size_check_name("G5M02M03M14",phi_G5M02M03M14,size)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
Total number of cases: 82
Empty: 0
Points: 0
Maximal polytopes: 82

 Number of cases of dim: -1:0, 0: 0, 1:0, 2:0, 3:0, 4:82, 5:0, 6:0, 7:0


## G6

In [8]:
size=6

In [ ]:
# size_check_name("G6",phi_G6,size)

In [31]:
size_check_name("G6M2",phi_G6M2,size)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
Total number of cases: 120
Empty: 0
Points: 0
Maximal polytopes: 120

 Number of cases of dim: -1:0, 0: 0, 1:0, 2:0, 3:0, 4:0, 5:120, 6:0, 7:0


In [33]:
size_check_name("G6M3",phi_G6M3,size)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
Total number of cases: 120
Empty: 0
Points: 0
Maximal polytopes: 120

 Number of cases of dim: -1:0, 0: 0, 1:0, 2:0, 3:0, 4:0, 5:120, 6:0, 7:0


Check with break divisors ? 

In [10]:
size_check_name("K33",phi_K33,size)

0
Total number of cases: 1
Empty: 0
Points: 0
Maximal polytopes: 1

 Number of cases of dim: -1:0, 0: 0, 1:0, 2:0, 3:0, 4:0, 5:1, 6:0, 7:0


In [11]:
size_check_name("K33_breaks",phi_K33,size)

0
Total number of cases: 1
Empty: 0
Points: 0
Maximal polytopes: 1

 Number of cases of dim: -1:0, 0: 0, 1:0, 2:0, 3:0, 4:0, 5:1, 6:0, 7:0


## Get others now

# Anaylysing the polytope data (for all case)

- data outputed as 
-Those where the average fails.

G62
Total number of cases: 48
Number of cases of dim:-1:0,0:0,1:0,2:0,3:0,4:0,5:48

G63 
Total number of cases: 36
Number of cases of dim: -1:0, 0: 0, 1:0, 2:0, 3:0, 4:0,5:36

G72
Total number of cases: 240
Number of cases of dim: -1:0, 0: 0, 1:0, 2:0, 3:0, 4:0, 5:0,6:240

G73
Total number of cases: 432
Number of cases of dim: -1:0, 0: 0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:432

recording the number of cases of a given dimesnion



# Functions

### Hyperplanes

### Analysis phi-polytope

In [7]:
def get_a_phi(poly):
#     phi=poly.center()
    phi=poly.representative_point()
    return phi

### Specific phi check satisfies

In [6]:
def mod_where_specific_lbm_fail(dict,lbm,phi):    
    """
    Checks if a given phi (not average phi) fails.
    
    Outputs where it fails or outputs f"The phi: {} does not fail".
    """
    
    def sum_comps(comp,indexor): #Done
        """returns the sum of the components of comp in the position by indexor"""
        #for d and phi

        sum_l=[comp[int(i)-1] for i in indexor]
        return sum(sum_l)

    def cutfrac(dict,indexor): #Done
        cutfrac=dict[indexor]
        return cutfrac
    
    counter=0
    for d in lbm:
        for indexor in dict.keys():
            if abs(sum_comps(d,indexor) - sum_comps(phi,indexor))<cutfrac(dict,indexor):
                pass
            else:
                print(f"Multidegree: {d}, \n Phi: {phi}, \n Indices summed {indexor}, \n Failed the inequality |{sum_comps(d,indexor)} - {sum_comps(phi,indexor)}|<{cutfrac(dict,indexor)} \n")
                counter+=1
                
    if counter !=0:
        x=f"The phi: {phi} fails the phi-inequalities"
    if counter ==0:
        x=f"The phi: {phi} does not fail the phi-inequalities"
    
    return x

In [25]:
def mod_get_formatted_hyperplanes_txt(lbm,phi_G): #put into format of sage b<Ax<b+1
    # Modified mod_where_specific_lbm_fail to include total degree equation and work for trees wrt phi inequalites for G62 G63 
    # not the same as phi inequal dicts for lbm.
    
    "Following doc phi-polytope inequalities"
    
    """
    Inputs:
    phi  records indexs of cuts as keys, and adjuster in modulus as value, Cf =0.5 in all cases. 
    lbm= line bundle mulidegrees given by chip-adding
    
    Purpose: Get list of inquality data to feed into Sages Polyhedron function
    https://doc.sagemath.org/html/en/reference/discrete_geometry/sage/geometry/polyhedron/constructor.html
    
    Polyhedron(ieqs=[(0,1,0),(0,0,1),(1,-1,-1)]).Hrepresentation()
    (An inequality (-1, -1) x + 1 >= 0,
     An inequality (1, 0) x + 0 >= 0,
     An inequality (0, 1) x + 0 >= 0)
     
    to add the total degree inequality x+y+z..=degree.
    add in ()
    
    """
    n=len(lbm[0])
    # We add the total degree equation to inequals
    deg=sum(lbm[0])
    tot_equ1=[-deg,]+ [1]*n
    tot_equ2=[deg,]+ [-1]*n
    
    inequals=[tot_equ1,tot_equ2]     
    for d in lbm: #creates row at a time
        for indexor in list(phi_G.keys()):
            
            def sum_comps(comp,indexor): #Done
                """returns the sum of the components of comp in the position by indexor"""
                #for d and phi
                sum_l=[comp[int(i)-1] for i in indexor]
                return sum(sum_l)
            
            cf=0.5 #for phi tree polytopes.
#             cf=phi_G[indexor] # want value
#             constant_indx= sum_comps(d,indexor)   
            constant_indx=phi_G[indexor] #for phi tree polytopes (remeber the minus) #!The adjustor.
        
            l_i= [ [int(x)-1,] for x in indexor] #x,y,z.. terms for indexors in Rf space #Indexs for phi keys
            

            def lower_part(constant_indx,l_i,indexor,cf,n):
                
                bterm=[cf-constant_indx] # as list
                base=np.zeros(n)
                l=[(1,i) for i in indexor] # put -1 in the ith position given be indexor.
                
                for item in l:
                    ipos = int(item[1])-1 # ith position
                    val=item[0]
                    base[ipos]=val
                
                xyz=base
                
                ineq=bterm+list(xyz) # [b,x,y,z...] want to concat                
                return ineq

            def upper_part(constant_indx,l_i,indexor,cf,n):
                
                #Remember upper part of modulus inequality.

                bterm=[cf+constant_indx] # as list
                base=np.zeros(n)
                l=[(-1,i) for i in indexor] # put -1 in the ith position given be indexor.
                
                for item in l:
                    ipos = int(item[1])-1 # ith position
                    val=item[0]
                    base[ipos]=val
                
                xyz=base
                
                ineq=bterm+list(xyz) # [b,x,y,z...] want to concat
                
                return ineq

            res_lower=lower_part(constant_indx,l_i,indexor,cf,n)
            
            # print("res_lower",res_lower)
            res_upper=upper_part(constant_indx,l_i,indexor,cf,n)
            
            # print("res_upper",res_upper)
            res_combined= [res_lower,res_upper]
            
            # print("--------")

            inequals=inequals+res_combined
    return inequals

# Later ------------------

## Get phi that works for Ibd case (phi center of polytope):

We construct the convex polytopal region for a stability condition $\sigma_{\Gamma}$. Denote this polytope by $R_\sigma^{\Gamma}$ for line bundle multidegrees.

We focus on the cases: $G62,G63,G72,G73, K_{33}$ and $GSym_3$

We aim to answer the following questions:

- For cases get Get random phi that works

## Get G455 center phi 

We get phi center for all lbm for G455 and complie G455_center_phi as a result.

In [ ]:
graphname="G4"
phi_G4={"1":1,"2":1,"3":1,"4":1,"12":1,"23":1}
G4_data=unpickle(graphname)

#Specific example
# G4_data[0] # ass, lbm as array
lbm=G4_data[0][1]
data=get_formatted_hyperplanes_txt(lbm,phi_G4)
tup=poly_size(data)
print(tup) #To determine size of data

# Check size for all G4 lbm examples
all_lbm=[i[1] for i in G4_data]

for lbm in all_lbm:
    data=get_formatted_hyperplanes_txt(lbm,phi_G4)
    tup=poly_size(data)
    print(tup) #To determine size of data
    
#     poly=Polyhedron(ieqs=data)
#     phi=get_a_phi(poly) # this is not the average
    # 
#     print(lbm)
#     mod_where_specific_lbm_fail(phi_G455,lbm,phi)
    
#     print("\n ----")

## G4M4 phi center specific example

In [8]:
"We take G455 and an example set of line bundle multidegrees."

lbm=np.array([[ 0.,  0.,  1.,  1.],
       [ 0.,  1.,  0.,  1.],
       [ 0.,  1.,  1.,  0.],
       [ 0.,  2.,  0.,  0.],
       [ 1.,  0.,  0.,  1.],
       [ 1.,  0.,  1.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  1., -1.]])

phi_G455={"1":1,"2":1.5,"3":1,"4":1.5,"12":1.5,"23":1.5,"34":1.5,"41":1.5}
data=get_formatted_hyperplanes_txt(lbm,phi_G455)
tup=poly_size(data)
# print(tup) #To determine size of data

poly=Polyhedron(ieqs=data)
phi=get_a_phi(poly)
print(phi)

# print(mod_where_specific_lbm_fail(phi_G455,lbm,phi))

(0.5, 0.75, 0.5, 0.25)


## G62

In [ ]:
pickle_in = open(r"lbm_data\G62_phi_failcases_lbm.txt", "rb")
data = pickle.load(pickle_in)
pickle_in.close()

In [ ]:
"We check if another phi works"

phi_G62={"1":1,"2":1.5,"3":1,"4":1,"5":1,"6":1.5,"12":1.5,"23":1.5,"34":1,"45":1,"56":1.5,"61":1.5,"123":1.5,"234":1.5,"345":1,"456":1.5,"561":1.5,"612":1}

lbm=data[0] # print(data[0]) #the first element is that of Integral break divisors.

hyperplanes=get_formatted_hyperplanes_txt(lbm,phi_G62)

poly=Polyhedron(ieqs=hyperplanes)
phi=get_a_phi(poly)

# print(mod_where_specific_lbm_fail(phi_G62,lbm,phi))

"""
G62
The phi (center of polytope): (1/6,2/3,1/6,1/6,1/6,2/3)  does not fail the phi-inequalities
phi average is: [0.35714286 0.5        0.21428571 0.21428571 0.21428571 0.5       ]
"""

## G63

In [ ]:
pickle_in = open(r"lbm_data\G63_phi_failcases_lbm.txt", "rb")
data = pickle.load(pickle_in)
pickle_in.close()

In [ ]:
"We check if another phi works"

lbm=data[0] # print(data[0]) #the first element is that of Integral break divisors.

phi_G63={"1":1,"2":1,"3":1.5,"4":1,"5":1,"6":1.5,"12":1,"23":1.5,"34":1.5,"45":1,"56":1.5,"61":1.5,"123":1.5,"234":1.5,"345":1.5,"456":1.5,"561":1.5,"612":1.5}

hyperplanes=get_formatted_hyperplanes_txt(lbm,phi_G63)

poly=Polyhedron(ieqs=hyperplanes)

phi=get_a_phi(poly)

# phi_av = [0.26666667 0.26666667 0.46666667 0.26666667 0.26666667 0.46666667]

mod_where_specific_lbm_fail(phi_G63,lbm,phi)

"""
G63
The phi (center of polytope): (1/6,1/6,2/3,1/6,1/6,2/3) does not fail the phi-inequalities
(0.16666666666666666, 0.16666666666666666, 0.6666666666666666, 0.16666666666666666, 0.16666666666666666, 0.6666666666666666) 
phi average is: [0.26666667 0.26666667 0.46666667 0.26666667 0.26666667 0.46666667]
"""

## Asides:

## Aside: Polytopes by phi inequalites for trees (0 assignment)

In [ ]:
- Obtain by hand for G62 and G63 get $R_\sigma^{\Gamma_i}$ (pdf). For G62 and G63:
    - Is $R_\sigma^{\Gamma}= R_\sigma^{\Gamma_i}$ and do they contain each other.
    - What are dim $R_\sigma^{\Gamma_i}$ are any polytopes are empty.

## G62 and Comparision with polytope by phi lbm inequalities (IBD)

In [30]:
#Load tree phi inequalities

G62_tree_phi_inequal=[{"2":1,"23":1,"234":1,"1":0.5,"2345":1},
{"3":0.5,"2":1,"21":1,"216":1.5,"2165":1.5},
{"4":0.5,"3":0.5,"32":1,"321":1,"3216":1.5},
{"5":0.5,"4":0.5,"43":0.5,"432":1,"4321":1},
{"6":1,"5":0.5,"54":0.5,"543":0.5,"5432":1},
{"6":1,"1":0.5,"12":1,"123":1,"1234":1},
{"1":0.5,"3":0.5,"34":0.5,"345":0.5,"3456":1},
{"1":0.5,"3":0.5,"4":0.5,"45":0.5,"456":1},
{"1":0.5,"5":0.5,"56":1,"4":0.5,"43":0.5},
{"1":0.5,"6":1,"5":0.5,"54":0.5,"543":0.5},
{"1":0.5,"2":1,"3":0.5,"34":0.5,"345":0.5},
{"1":0.5,"3":0.5,"32":1,"4":0.5,"45":0.5},
{"1":0.5,"4":0.5,"43":0.5,"432":1,"5":0.5},
{"1":0.5,"5":0.5,"54":0.5,"543":0.5,"5432":1}]

In [31]:
#Compare each poly_tree to poly lbm for Ibd

pickle_in = open(r"lbm_data\G62_phi_failcases_lbm.txt", "rb")
data = pickle.load(pickle_in)
pickle_in.close()

phi_G62={"1":1,"2":1.5,"3":1,"4":1,"5":1,"6":1.5,"12":1.5,"23":1.5,"34":1,"45":1,"56":1.5,"61":1.5,"123":1.5,"234":1.5,"345":1,"456":1.5,"561":1.5,"612":1}
lbm=data[0] # print(data[0]) #the first element is that of Integral break divisors.
hyperplanes=get_formatted_hyperplanes_txt(lbm,phi_G62)
G62poly=Polyhedron(ieqs=hyperplanes)

In [32]:
#Properties to distinguish polyhedra

# G62poly.f_vector() 
# G62poly.n_vertices() #6
# G62poly.radius() #0.9128709291752769
# G62poly.representative_point()

#(0.16666666666666666, 0.6666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.6666666666666666)
trans=(-0.16666666666666666, -0.6666666666666666,- 0.16666666666666666, -0.16666666666666666,- 0.16666666666666666, -0.6666666666666666)
transG62poly=G62poly.translation(trans)

pt=transG62poly.representative_point()
pt=[round(i,5) for i in pt]
print(pt)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [33]:
dim_check=[]
pols=[]

print("f-vector G62poly",G62poly.f_vector(),"\n")

for inequ in G62_tree_phi_inequal:
    lbm=[[0,0,0,0,0,0]]
    hyperp=mod_get_formatted_hyperplanes_txt(lbm,inequ)
    tree_poly=Polyhedron(ieqs=hyperp)
    pols.append(tree_poly)
    
#     for i in tree_poly.Hrepresentation():
#         print(i)
    
#     print(dim(G62poly&tree_poly)) #in different hyperplanes. #https://ask.sagemath.org/question/26602/can-i-intersect-the-boundaries-of-two-polyhedra-and-display-it/
    print("Are they iso?",G62poly.is_combinatorially_isomorphic(tree_poly))
    print("f-vector",tree_poly.f_vector()) #Are these polytopes the same?
#     print(tree_poly.n_vertices()) #32
#     print(tree_poly.radius())
    x=tree_poly &transG62poly #Do not contain.
    print("Do they contain each other?",dim(x))
    
    print("rep point",tree_poly.representative_point())
    
#     if x.is_combinatorially_isomorphic(transG62poly):
#         print("contains")
#     else:
#         print("doesnt contain")

    dimp=poly_size(hyperp)
    print("dimension",dimp)
    dim_check.append(dimp)
    print("--------")
print(dim_check) # [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]



f-vector G62poly (1, 6, 15, 20, 15, 6, 1)
Are they iso? False
f-vector (1, 32, 80, 80, 40, 10, 1)
Do they contain each other? -1
rep point (0.5, 1.0, 0.0, 0.0, 0.0, -1.5)
dimesnsion 5
--------
Are they iso? False
f-vector (1, 32, 80, 80, 40, 10, 1)
Do they contain each other? -1
rep point (0.0, 1.0, 0.5, -2.0, 0.0, 0.5)
dimesnsion 5
--------
Are they iso? False
f-vector (1, 32, 80, 80, 40, 10, 1)
Do they contain each other? -1
rep point (0.0, 0.5, 0.5, 0.5, -2.0, 0.5)
dimesnsion 5
--------
Are they iso? False
f-vector (1, 32, 80, 80, 40, 10, 1)
Do they contain each other? -1
rep point (0.0, 0.5, 0.0, 0.5, 0.5, -1.5)
dimesnsion 5
--------
Are they iso? False
f-vector (1, 32, 80, 80, 40, 10, 1)
Do they contain each other? -1
rep point (-2.0, 0.5, 0.0, 0.0, 0.5, 1.0)
dimesnsion 5
--------
Are they iso? False
f-vector (1, 32, 80, 80, 40, 10, 1)
Do they contain each other? -1
rep point (0.5, 0.5, 0.0, 0.0, -2.0, 1.0)
dimesnsion 5
--------
Are they iso? False
f-vector (1, 32, 80, 80, 40, 10,

In [ ]:
#Comparing phi-tree-polys:

# for p in pols:
#     for q in pols:
#         if p==q:
#             pass
#         else:
# #             print(p==q) #Not equal
# #             print(p.is_combinatorially_isomorphic(q)) #are isomorphic to each other.
#     print("finished this p")

# #Taking intersection of phi-tree-polys
# q=pols[0]
# for p in pols:
#     q=q & p
# print(dim(q)) #5

#Finding a representive point
# r=Polyhedron(ieqs=[(-2,1,1,1,1,1,1),(2,-1,-1,-1,-1,-1,-1)])
# z=q&r
# pt=z.representative_point()
# print(pt)

## G63  and Comparision with polytope by phi lbm inequalities (IBD)

In [ ]:
G63_tree_phi_inequal=[{"1":0.5,"2":0.5,"23":1,"234":1,"2345":1},
{"2":0.5,"3":1,"34":1,"345":1,"3456":1.5},
{"3":1,"4":0.5,"45":0.5,"456":1,"4561":1},
{"4":0.5,"5":0.5,"56":1,"561":1,"5612":1},
{"5":0.5,"6":1,"61":1,"612":1,"6123":1.5},
{"6":1,"1":0.5,"12":0.5,"123":1,"1234":1},
{"4":0.5,"1":0.5,"12":0.5,"123":1,"1236":1.5},
{"4":0.5,"5":0.5,"1":0.5,"12":0.5,"56":1},
{"1":0.5,"12":0.5,"5":0.5,"54":0.5,"6":1},
{"1":0.5,"2":0.5,"23":1,"4":0.5,"45":0.5},
{"2":0.5,"1":0.5,"4":0.5,"5":0.5,"234":0.5},
{"1":0.5,"16":1,"2":0.5,"5":0.5,"54":0.5},
{"2":0.5,"21":0.5,"4":0.5,"45":0.5,"3":1},
{"2":0.5,"21":0.5,"4":0.5,"43":1,"5":0.5},
{"2":0.5,"21":0.5,"216":1,"2163":1.5,"5":0.5}]

In [ ]:
dim_check=[]
for inequ in G63_tree_phi_inequal:
    lbm=[[0,0,0,0,0,0]]
    hyperp=mod_get_formatted_hyperplanes_txt(lbm,inequ)
    dim_check.append(poly_size(hyperp))
print(dim_check)
# [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

## K33 testing dimension of phi-tree-polytope for selected trees

In [41]:
#We choose a selection of trees and determine the dimension of the polytope.

K33_tree_phi_inequal=[{"6":1,"1":1,"12":1.5,"123":2,"1234":2.5},#from necklace
                     {"2":1,"3":1,"25":1.5,"254":2,"2541":2.5,}] #not necklace tree
    
dim_check=[]
for inequ in K33_tree_phi_inequal:
    lbm=[[0,0,0,0,0,0]]
    hyperp=mod_get_formatted_hyperplanes_txt(lbm,inequ)
    
#     for i in hyperp: #Work and cor to paper calc
#         print(i)
#     print("\n")
    
    dim_check.append(poly_size(hyperp))
    
    tree_poly=Polyhedron(ieqs=hyperp)
    
#     for i in tree_poly.Hrepresentation():
#         print(i)
    
#     print("\n rep point",tree_poly.representative_point())
    
    
print(dim_check)


[5, 5]


## $GSym_3$

In [35]:
Gsym3_tree_phi_inequal=[{"1":1,"12":1.5,"123":2}] #from necklace tree
    
dim_check=[]
for inequ in Gsym3_tree_phi_inequal:
    lbm=[[0,0,0,0]]
    hyperp=mod_get_formatted_hyperplanes_txt(lbm,inequ)
    dim_check.append(poly_size(hyperp))
print(dim_check)


[3]


# LEFTOVERS

In [ ]:
graphname="G4"
phi_G4={"1":1,"2":1,"3":1,"4":1,"12":1,"23":1}
G4_data=unpickle(graphname)

# #Specific example
# # G4_data[0] # ass, lbm as array
# lbm=G4_data[0][1]
# data=get_formatted_hyperplanes_txt(lbm,phi_G4)
# tup=poly_size(data)
# print(tup) #To determine size of data

# Check size for all G4 lbm examples
all_lbm=[i[1] for i in G4_data]


# for lbm in all_lbm:
#     data=get_formatted_hyperplanes_txt(lbm,phi_G4)
#     tup=poly_size(data)
#     print(tup) #To determine size of data

In [ ]:
# graphname="G4"
# phi_G4={"1":1,"2":1,"3":1,"4":1,"12":1,"23":1}
# G4_data=unpickle(graphname)

# # Determine size of phi-polytopes
# all_lbm=[i[1] for i in G4_data]
# polytope_size_checker(all_lbm,phi_G4,4)